# 測試多模態模型提取 PDF 中需要欄位：

In [2]:
# 處理 Text to JSON
import json
import re
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

load_dotenv()
project_id = "nthurc-aisearch-202406"
model_name = "gemini-1.5-flash-001"

def parse_json_from_text(response_text):
    """
    從包含 JSON 或 JSON Lines 的文字中解析 JSON 資料。

    Args:
        response_text: 包含 JSON 或 JSON Lines 資料的字串。

    Returns:
        一個包含解析後的 JSON 資料的列表。
    """

    # 使用正則表達式去除開頭的非 JSON 字元
    json_strings = re.sub(r'^[^\{]+', '', response_text).rstrip('`').strip().split('\n')

    data = []
    for json_string in json_strings:
        # 檢查是否為空字串，若是則跳過
        if not json_string.strip():
            continue

        try:
            data.append(json.loads(json_string))
        except json.JSONDecodeError as e:
            print(json_string)
            print(f"Error decoding JSON: {e}")

    return data

def process_taxpayer_document(project_id, model_name,part, prompt):
    vertexai.init(project=project_id, location="asia-east1")
    model = GenerativeModel(model_name)
    
    contents = [part, prompt]
    response = model.generate_content(contents)
    
    print(response.text)
    print("=============")
    print("處理過後的 JSON : \n\n", parse_json_from_text(response.text))
    
    return parse_json_from_text(response.text)

### 1. 所得相關資料提取

- 目前使用 Vertex AI ( Gemini-1.5-flash 多模態模型 ) 來提取 PDF 中的資料 
- 明確定義我們需要的提取的內容
- 定義 JSON
- 設計 Prompt 並讓 Gemini 理解任務
- 去得輸出，並改成可解析的結構

#### 這部分目前需要釐清的問題 : 
- 電子財產：這部分的檔案是否也要在這一塊進行處理？
- 這一部分的檔案除了目前欄位以外是否還有其他欄位有需求

In [3]:
json_format = '''{
  "taxYear": 提取此所得資料清單[年度]資訊，位於文檔頂部,
  "IssuingAuthority": 提取此所得稅[核發單位]文字資訊放於此欄位,
  "taxpayerName": 提取[所得人姓名],
  "taxpayerID": 提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼,
  "reportDate": 提取此報告[查調日期]，位於文檔右上角,
  "totalIncome": 提取此所得人總所得，位於文檔下方有一個[給付總額合計]的資訊,
}'''

prompt = f"""
You are now a professional PDF document extractor. I will provide you with a property income document from the Ministry of Finance of Taiwan. You will extract the information from each page and organize it into JSONL to return to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSONL :
"""

taxpay_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/所得/所得14.pdf"
taxpay_pdf_file = Part.from_uri(taxpay_pdf_file_uri, mime_type="application/pdf")
json = process_taxpayer_document(project_id, model_name, taxpay_pdf_file, prompt)
print(json)

```json
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃德旺", "taxpayerID": "R124075657", "reportDate": "111年07月11日", "totalIncome": "724,894"}
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃春萬", "taxpayerID": "R120335830", "reportDate": "111年07月11日", "totalIncome": "107"}
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃麗珍", "taxpayerID": "Q221609858", "reportDate": "111年07月11日", "totalIncome": "0"}
```
處理過後的 JSON : 

 [{'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃德旺', 'taxpayerID': 'R124075657', 'reportDate': '111年07月11日', 'totalIncome': '724,894'}, {'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃春萬', 'taxpayerID': 'R120335830', 'reportDate': '111年07月11日', 'totalIncome': '107'}, {'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃麗珍', 'taxpayerID': 'Q221609858', 'reportDate': '111年07月11日', 'totalIncome': '0'}]
('```json\n{"tax

### 電子所得測試

In [44]:
etaxpay_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/所得/電子所得23.pdf"
etaxpay_pdf_file = Part.from_uri(etaxpay_pdf_file_uri, mime_type="application/pdf")
contents = [etaxpay_pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

print("=============")
print("處理過後的 JSON : \n\n",parse_json_from_text(response.text))

```jsonl
{"taxYear": "110年度", "IssuingAuthority": "財政部北區國稅局板橋分局", "taxpayerName": "徐孟伶", "taxpayerID": "H222398161", "reportDate": "111年06月07日", "totalIncome": "857,326"}
{"taxYear": "110年度", "IssuingAuthority": "財政部北區國稅局板橋分局", "taxpayerName": "陳建良", "taxpayerID": "T122820496", "reportDate": "111年06月07日", "totalIncome": "386, 631"}
{"taxYear": "110年度", "IssuingAuthority": "財政部北區國稅局板橋分局", "taxpayerName": "陳心瞳", "taxpayerID": "F233576482", "reportDate": "111年06月07日", "totalIncome": "0"}
{"taxYear": "110年度", "IssuingAuthority": "財政部北區國稅局板橋分局", "taxpayerName": "陳方素雲", "taxpayerID": "T201699868", "reportDate": "111年06月07日", "totalIncome": "0"}
{"taxYear": "110年度", "IssuingAuthority": "財政部北區國稅局板橋分局", "taxpayerName": "陳訢恩", "taxpayerID": "F133800696", "reportDate": "111年06月07日", "totalIncome": "0"}
```
處理過後的 JSON : 

 [{'taxYear': '110年度', 'IssuingAuthority': '財政部北區國稅局板橋分局', 'taxpayerName': '徐孟伶', 'taxpayerID': 'H222398161', 'reportDate': '111年06月07日', 'totalIncome': '857,326'}, {'taxYear': '

## 2. 戶籍謄本相關資料提取
- 跟上一個方法一樣，通過定義需要的欄位後,讓多模態模型進行視覺的資料提取並轉換成文字回傳

#### 這部分需要釐清的部分 : 
- 戶籍謄本帶現戶全戶這個後綴，是需要提取還是判斷是否有這個欄位？
- 提取這個現戶全戶的目的是什麼，為了區分戶口名簿嗎？
- 是否只需要提取戶長資訊即可？

In [46]:
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

project_id = "nthurc-aisearch-202406"

load_dotenv()
vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

json_format = '''{
  "IssueDate" : 取得此 PDF 中[列印日期/時間]中的日期部分,
  "IsFamilyRegisterExtract" :  判斷[戶籍謄本]後是否帶有(現戶全戶)，如果有返回 True, 否則返回 False, 如果前綴是戶口名則輸出 None,
  "householdId" : 取得此 PDF 中的[戶號],
  "householdAddress" : 取得此 PDF 中的[戶籍地址],
  "headOfHousehold" : 判斷此 PDF 中[戶長]相關資訊並組合成一個嵌套的 JSON {
    "name" : 取得此戶長的姓名,
    "birthDate" : 取得此戶長的出生日期,
    "PersonID" : 取得此戶長的身份證字號,
    "father" : 取得此戶長的父親姓名,
    "mother" : 取得此戶長的母親姓名,
    "note" : 取得此戶長的[記事]內所有紀錄的資訊
  }
}'''


prompt = f"""
You are now a professional PDF document extractor. I will provide you with a PDF of a household registration book or household registration transcript provided by the Taiwan government. You will extract the specified information from the PDF to JSON and then return it to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSON :
"""

FamilyRegisterExtract_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶謄13.pdf"
FamilyRegisterExtract_pdf_file = Part.from_uri(FamilyRegisterExtract_pdf_file_uri, mime_type="application/pdf")
contents = [FamilyRegisterExtract_pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

```json
{
  "IssueDate": "111/07/04",
  "IsFamilyRegisterExtract": true,
  "householdId": "FR298901",
  "householdAddress": "新北市永和區得和里022鄰中正路368巷20弄4號三樓",
  "headOfHousehold": {
    "name": "鍾照明",
    "birthDate": "民國60年4月20日",
    "PersonID": "V120407442",
    "father": "范先開",
    "mother": "鍾春桂",
    "note": "原住台東縣台東市建業里5鄰西昌街242號鍾春桂戶內民國84年6月17日遷入。民國86年3月29日與陳雪如結婚民國86年4月7日申登。民國94年3月16日註記原住民民族別。原住臺北市大同區延平里14鄰重慶北路二段114巷11號五樓陳宗隆戶內民國97年9月26日遷入登記"
  }
}
```


### 戶口名簿測試

In [47]:
HouseholdRegistry_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/戶籍/戶口7.pdf"
HouseholdRegistry_file = Part.from_uri(HouseholdRegistry_file_uri, mime_type="application/pdf")
contents = [HouseholdRegistry_file, prompt]

response = model.generate_content(contents)
print(response.text)

```json
{
  "IssueDate": "111/07/06",
  "IsFamilyRegisterExtract": True,
  "householdId": "S5641954",
  "householdAddress": "新北市樹林區中山里025鄰中山路三段175號十一樓之5",
  "headOfHousehold": {
    "name": "黃志禾",
    "birthDate": "民國70年3月23日",
    "PersonID": "E122398826",
    "father": "黃明俊",
    "mother": "莊偉琍",
    "note": "原住高雄縣鳳山市文山里21鄰文中街62巷1弄23之3號黃明俊戶內民國84年5月9日遷入。民85年3月5日憑通報註記失蹤人口案號E05426。民國85年5月30日撤銷失蹤民國85年6月29日登記。原住高雄市鼓山區麗興里10鄰捷興二街7之2號楊建治戶內民國86年10月14田遷人。民國94年2月26日與陳念慈結婚民國94年3月2日車登。長男黃婕瑞統一編號S126129131民國99年3月26日出生(S1)。民國102年5月20日與陳念慈兩願離婚。原登記母姓名莊淑洪因母改名民國107年4月23日變更民國107年4月23日經臺北市士林區戶政事務所逕為變更登記。民國109年4月8日行使負擔未成年子女黃葳琦權利義務民國109年4月28日申登·民國109年4月8日行使負擔未成年子女黃婕瑞權利義務民國109年4月28日申登。"
  }
}
```


## 3. 財產查詢清單

#### 目前需要釐清的部分：
- 財產中只有土地需要被紀錄嗎？是需要被紀錄是否持有還是有其他資訊也需要讀取到？
- 財產中如果有房產資訊，是否需要紀錄房產的資訊？

In [8]:
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

project_id = "nthurc-aisearch-202406"

load_dotenv()
vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

json_format = '''{
  "IssuingAuthority": 提取此所得稅[核發單位]文字資訊放於此欄位,
  "AssetName": 提取[所得人姓名],
  "AssetID": 提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼,
  "HasLand" : 如果財產列表中有土地相關的財產，請返回 True, 否則返回 False
}'''


prompt = f"""
You are now a professional PDF document extractor. I will provide you with a PDF of a Asset Inquiry List provided by the Taiwan government. You will extract the specified information from the PDF to JSONL and then return it to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSONL :
"""

AssetInquiryList_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/財產/財產22.pdf"
AssetInquiryList_pdf_file = Part.from_uri(AssetInquiryList_pdf_file_uri, mime_type="application/pdf")
contents = [AssetInquiryList_pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

```jsonl
{"IssuingAuthority": "財政部北區國稅局板橋分局", "AssetName": "廖庭霙", "AssetID": "Y220260353", "HasLand": false}
{"IssuingAuthority": "財政部北區國稅局板橋分局", "AssetName": "廖欽日", "AssetID": "A103299724", "HasLand": false}
{"IssuingAuthority": "財政部北區國稅局板橋分局", "AssetName": "陳宣卉", "AssetID": "F228511773", "HasLand": false}
{"IssuingAuthority": "財政部北區國稅局板橋分局", "AssetName": "廖楊金魚", "AssetID": "A203144271", "HasLand": false}
```


## 4. 社宅申請表
#### 需要討論的問題
- 手寫文件、是否要通過
- 是否只需要基本資訊 : 如申請人基本資料、申請項目等相關資訊
- 是否只需要他勾選了什麼的資訊即可，為勾選的則留空白 

In [21]:
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

project_id = "nthurc-aisearch-202406"

load_dotenv()
vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

json_format = '''{
    "receiveDate" : 提取右上角[收件日期]欄位的手寫日期,
    "receiveNumber" : 提取右上角[收件編號]欄位的手寫編號,
    "applicantName": 提取表格上方中間申請人姓名的手寫內容，位於「本人」後面,
    "applicationDate": 提取表格中間右側[申請日期]欄位的手寫日期,
    "applicantBasics" : 一、申請人基本資料表{
        "ApplicantName": 提取[申請人姓名]欄位的手寫內容，位於表格左上角,
        "Gender": 提取性別欄位的勾選狀態，男/女，位於姓名欄位右側,
        "DateOfBirth": 提取[出生年月日]欄位的手寫內容，格式為民國年/月/日，位於性別欄位右側,
        "MaritalStatus": 提取婚姻欄位的勾選狀態，已婚/未婚，位於出生年月日欄位右側,
        "IDNumber": "提取[身分證統一編號]欄位的手寫內容，位於表格第二行左側",
        "HouseholdRegistrationNumber": "提取[戶口名簿戶號]欄位的手寫內容，位於表格第二行右側",
        "ContactPhone": "提取[聯絡電話]欄位的手寫內容，位於表格第三行左側",
        "MobilePhone": "提取[行動電話]欄位的手寫內容，位於表格第三行右側",
        "Email": "提取[電子郵件信箱]欄位的手寫內容，位於表格第四行",
        "RegisteredAddress": "提取[戶籍地址]欄位的手寫內容，包括郵遞區號、縣市、鄉鎮市區等信息，位於表格第五行",
        "MailingAddress": "提取[通訊地址]欄位的勾選狀態（同上）和可能的手寫內容，位於表格第六行",
        "EmergencyContact": "提取[緊急聯絡人]欄位的手寫內容，位於表格最後一行左側",
        "EmergencyContactPhone": "提取緊急聯絡人的[行動電話]欄位的手寫內容，位於表格最後一行右側"
        },
    "applicationItems": 二、申請項目表格{
        "ApplicantHousingType": "提取[申請房型]欄位的勾選狀態，位於表格左上方，包括一房/套房、二房、三房等選項",
        "ApplicantLocation": "提取[申請社區]欄位的勾選狀態(可複選，以列表呈現)，位於表格右上方，包括永和中正橋、板橋永翠等選項",
        "HouseholdType": "提取[一般戶]欄位下方的勾選狀態(可複選，以列表呈現)，位於表格中間偏左，包括一般青年戶、優先戶等選項",
        "PriorityGroup": "提取[優先戶]欄位下方的勾選狀態(可複選，以列表呈現)，位於表格下半部分，包括低收入戶、中低收入戶、特殊境遇家庭等選項",
        "DisabilityStatus": "提取[身心障礙者]欄位的勾選狀態，位於表格左下方",
        "SingleParentFamily": "提取[單親家庭]欄位的勾選狀態，位於表格左下方",
        "ViolenceVictim": "提取[家庭暴力或性侵害受害者及其子女]欄位的勾選狀態，位於表格左下方",
        "DisasterVictim": "提取[重大災害災民]欄位的勾選狀態，位於表格中下方",
        "ElderlyStatus": "提取[老人]欄位下方的勾選狀態，位於表格右下方，包括65歲以上、75歲以上等選項",
        "IndigenousStatus": "提取[原住民]欄位的勾選狀態，位於表格右下方",
        "OverseasChineseStatus": "提取[僑胞]欄位的勾選狀態，位於表格右下方",
        "EnvironmentallyDisplacedPerson": "提取[因環境災害或其他原因遷居者]欄位的勾選狀態，位於表格最下方",
    },
    "familyMembers":三、家庭成員資料{
        "familyMember": {
            "FamilyMemberName": 提取所有該家庭的成員，並順序輸出其姓名，並取得該人員詳細資料,
            "IDNumber": "提取[國民身分證統一編號/居留證統一證號]欄位的手寫內容，位於表格左側第二列",
            "Relationship": "提取[稱謂]欄位的手寫內容，位於表格右側第三列",
            "AnnualIncome": "提取[最近一年所得]欄位的手寫內容，位於表格最右側一列",
        },
        "TotalFamilyIncome": "提取[家戶最近一年總所得合計]欄位的手寫內容，位於表格底部倒數第二行",
        "AverageMonthlyIncome": "提取[平均每人每月所得]欄位的手寫內容，位於表格底部最後一行"
    }
}'''


prompt = f"""
You are now a professional PDF document extractor. I will provide you with a PDF of a Asset Inquiry List provided by the Taiwan government. You will extract the specified information from the PDF to JSONL and then return it to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSONL :
"""

AssetInquiryList_pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/申請書/申請書2.pdf"
AssetInquiryList_pdf_file = Part.from_uri(AssetInquiryList_pdf_file_uri, mime_type="application/pdf")
contents = [AssetInquiryList_pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)

```json
{"receiveDate": "111.7.16", "receiveNumber": "YHC0100", "applicantName": "段榮昌", "applicationDate": "111年7月16日", "applicantBasics": {"ApplicantName": "段榮昌", "Gender": "男", "DateOfBirth": "70年12月7日", "MaritalStatus": "已婚", "IDNumber": "F125437447", "HouseholdRegistrationNumber": "FOA37QF4", "ContactPhone": "(02)22183748", "MobilePhone": "0905211535", "Email": "f7447kim0@yahoo.com.tw", "RegisteredAddress": "新北市(縣)中和區連坊里(村)18鄰連城路(街)段171巷弄30號7樓之", "MailingAddress": "同上", "EmergencyContact": "林雅甄", "EmergencyContactPhone": "0911590660"}, "applicationItems": {"ApplicantHousingType": "二房", "ApplicantLocation": ["板橋永翠"], "HouseholdType": ["一般青年戶"], "PriorityGroup": ["中低收入戶"], "DisabilityStatus": null, "SingleParentFamily": null, "ViolenceVictim": null, "DisasterVictim": null, "ElderlyStatus": null, "IndigenousStatus": null, "OverseasChineseStatus": null, "EnvironmentallyDisplacedPerson": null}, "familyMembers": {"familyMember": [{"FamilyMemberName": "段榮昌", "IDNumber": "F125437447", "Rel